***
# Car prices
***

1. Répertorier l'ensemble des marques de véhicules
2. Répertorier l'ensemble des modèles de véhicules
3. Répertorier l'ensemble des prix des véhicules
***

In [19]:
# Create folders and getting path
import os
cwd = os.getcwd()

# Loop to create folders
folder_names = ['Dataframe', 'Output', 'Data']

folders = {}
for folder_name in folder_names:
    folders[folder_name] = os.path.join(cwd, folder_name)

    if not os.path.exists(os.path.join(cwd, folder_name)):
        os.makedirs(os.path.join(cwd, folder_name))
        print(f'Le dossier « {folder_name} » a été créé')

    else:
        print(f'Le dossier « {folder_name} » est existant')


# Création des variables de path

# Déterminer si os est win ou linux pour définir les path
if os.name == 'nt':
    slash = '\\'
elif os.name == 'posix':
    slash = '/'

path_prog =     cwd + slash
path_data =     folders['Data'] + slash
path_df =       folders['Dataframe'] + slash
path_output =   folders['Output'] + slash

# Mettre \\ pour éviter les erreurs
path_dict = [path_prog, path_data, path_df, path_output]
for path in path_dict:
    path = path.replace('\\','\\\\')


# Détermination de l'année
print()
print(path_prog)
print(path_data)
print(path_df)
print(path_output)

/home/charles/Documents/GitHub/car_sales_predictions/
/home/charles/Documents/GitHub/car_sales_predictions/Data/
/home/charles/Documents/GitHub/car_sales_predictions/Dataframe/
/home/charles/Documents/GitHub/car_sales_predictions/Output/


In [6]:
import requests
import json

response_api = requests.get('https://www.guideautoweb.com/constructeurs/')
print(response_api.status_code)     # 200 is ok

200


In [10]:
# Get the data from api
data = response_api.text

In [13]:
# Convert Html to json
from bs4 import BeautifulSoup
soup = BeautifulSoup(data, 'html.parser')

In [17]:
soup

<!DOCTYPE html>

<html lang="fr">
<head>
<meta charset="utf-8"/>
<title>Constructeurs automobiles - Guide Auto</title>
<script>var lq=lq||[];(function(){jskit=document.createElement('script');jskit.src='/js/c/jquery.3,cookie,url,cssua,lazyload,lcm,lcm.nots,lcm.vid,pads,g17+main,g17+modernizr,1612291859';jskit.async=true;jskit.onload=function(){while(lq.length){try{lq.shift()()}catch(e){}}lq={push:function(a){a()}}};document.head.appendChild(jskit)})();</script>
<script>lq.push(function(){jQuery(document).on('click','a[href]:not(.no-popup)',function(){if(/^https?:\/\/(?!(?:[^.]+\.)*www.guideautoweb.com)/.test(jQuery(this).attr('href')))jQuery(this).attr('target','_blank')})});</script>
<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src='https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);})(window,docume

In [ ]:
# Extract list of all models
# Extract list of web links to the models

In [ ]:
# Extract list of 'model en production'
# Extract web link to those models

In [ ]:
# Extract price and fuel consomption for every model